<a href="https://colab.research.google.com/github/pawelchlastacz/dw_matrix/blob/master/Dzien5ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install eli5

     |████████████████████████████████| 112kB 2.8MB/s 


In [0]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance

from ast import literal_eval
from tqdm import tqdm_notebook


In [2]:
cd "/content/drive/My Drive/Colab Notebooks/dw_matrix"

/content/drive/My Drive/Colab Notebooks/dw_matrix


In [4]:
ls data


men_shoes.csv  shoes_prices.csv


In [0]:
df = pd.read_csv('data/men_shoes.csv', low_memory=False)

In [0]:
def run_model(feats, model = DecisionTreeRegressor(max_depth=5)):
  X = df[ feats ].values
  y = df['prices_amountmin'].values

  scores = cross_val_score(model, X, y, scoring= 'neg_mean_absolute_error')
  return np.mean(scores), np.std(scores)

 

In [25]:
df  ['brand_cat'] = df["brand"].map(lambda x: str(x).lower()).factorize()[0]
run_model(['brand_cat'])

(-58.133398968282776, 4.206122611474276)

In [26]:
 model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
 run_model(['brand_cat'], model)

(-57.31783843165656, 4.181246596160967)

In [24]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_Cat,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"

In [28]:
df.features.head().values

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SER

In [29]:
test = {'key': 'value'}
test ['key']

'value'

In [32]:
df.head()

,id,asins,brand,categories,colors,count,dateadded,dateupdated,descriptions,dimension,ean,features,flavors,imageurls,isbn,keys,manufacturer,manufacturernumber,merchants,name,prices_amountmin,prices_amountmax,prices_availability,prices_color,prices_condition,prices_count,prices_currency,prices_dateadded,prices_dateseen,prices_flavor,prices_issale,prices_merchant,prices_offer,prices_returnpolicy,prices_shipping,prices_size,prices_source,prices_sourceurls,prices_warranty,quantities,reviews,sizes,skus,sourceurls,upc,vin,websiteids,weight,brand_Cat,brand_cat
0,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",39.89,39.89,NaN,NaN,NaN,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,True,NaN,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,0
1,AVpfHrJ6ilAPnD_xVXOI,NaN,Josmo,"Clothing,Shoes,Men's Shoes,All Men's Shoes",NaN,NaN,2016-11-07T00:45:12Z,2016-11-07T00:45:12Z,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""sourceU...",NaN,6.993020e+11,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Shoe...",NaN,https://i5.walmartimages.com/asr/13ac3d61-003c...,NaN,"josmo/8190wnavy75,699302044036,0699302044036",NaN,8190-W-NAVY-7.5,"[{""dateSeen"":[""2016-11-07T00:45:12Z""],""name"":""...","Josmo 8190 Plain Infant Walking Shoes, Navy - ...",51.99,51.99,NaN,NaN,new,NaN,USD,2016-11-07T00:45:12Z,2016-11-05T00:00:00Z,NaN,False,UnbeatableSale - Walmart.com,REDUCED USD 12.10,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,NaN,NaN,NaN,NaN,NaN,https://www.walmart.com/ip/Josmo-8190-Plain-In...,6.993020e+11,NaN,NaN,NaN,0,0
2,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,40.02,40.02,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2016-03-08T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,1
3,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"[{""dateSeen"":[""2016-06-14T04:29:57Z""],""name"":""...",Servus By Honeywell Shoe Studs Zsr101blmlg,50.31,50.31,NaN,NaN,new,NaN,USD,2016-06-14T04:29:57Z,2015-11-30T00:00:00Z,NaN,False,SIM Supply Inc - Walmart.com,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,NaN,http://www.walmart.com/ip/Studs-Shoe-Large-Pr-...,NaN,NaN,NaN,NaN,1,1
4,AVpfHsWP1cnluZ0-eVZ7,NaN,SERVUS BY HONEYWELL,"All Men's Shoes,Shoes,Men's Shoes,Clothing",NaN,NaN,2016-06-14T04:29:57Z,2016-07-09T20:26:48Z,"[{""dateSeen"":[""2016-07-09T20:26:48Z""],""sourceU...",NaN,NaN,"[{""key"":""Gender"",""value"":[""Men""]},{""key"":""Colo...",NaN,http://i5.walmartimages.com/dfw/dce07b8c-5844/...,NaN,servusbyhoneywell/zsr101blmlg,NaN,ZSR101BLMLG,"

In [0]:
def parse_features(x):
  output_dict = {}
  if str(x)  == 'nan': return output_dict

  features = literal_eval(x.replace('\\"', '"'))
  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dict[key]=value
  
  return output_dict
  
df['features_parsed'] = df['features'].map(parse_features)

In [56]:
keys = set()

df['features_parsed'].map( lambda x: keys.update(x.keys()) )

len(keys)

476

In [0]:

#[
# {'key': 'Gender', 'value': ['Men']}, 
# {'key': 'Shoe Size', 'value': ['M']}, 
# {'key': 'Shoe Category', 'value': ["Men's Shoes"]}, 
# {'key': 'Color', 'value': ['Multicolor']}, 
# {'key': 'Manufacturer Part Number', 'value': ['8190-W-NAVY-7.5']}, 
# {'key': 'Brand', 'value': ['Josmo']}]

# {
#     'Gender': 'Men'
#     ' Shoe Size': 'M'
# }



In [58]:
def get_name_feat(key):
  return 'feat_' + key
 
for key in tqdm_notebook(keys):
  
  df[get_name_feat(key)] = df.features_parsed.map(lambda feats: feats[key] if key in feats else np.nan)

In [57]:
df.features_parsed.head().values

array([{'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'shoe size': 'm', 'shoe category': "men's shoes", 'color': 'multicolor', 'manufacturer part number': '8190-w-navy-7.5', 'brand': 'josmo'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'},
       {'gender': 'men', 'color': 'black', 'shipping weight (in pounds)': '0.45', 'condition': 'new', 'brand': 'servus by honeywell', 'manufacturer_part_number': 'zsr101blmlg'}],
      dtype=object)

In [59]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_smart watch', 'feat_parts', 'feat_is recyclable',
       'feat_clothing category', 'feat_part type', 'feat_work shoes',
       'feat_case thickness', 'feat_dial color', 'feat_weight', 'feat_bridge'],
      dtype='object', length=527)

In [0]:
keys_stat = {}
for key in keys:
  keys_stat[key] =df [False == df['feat_athlete'].isnull() ].shape[0] / df.shape[0] *100

In [79]:
{k:v for k,v in keys_stat.items() if v> 40} 

{}

In [0]:
df  ['feat_brand_cat'] = df["feat_brand"].factorize()[0]
df  ['feat_color_cat'] = df["feat_color"].factorize()[0]
df  ['feat_gender_cat'] = df["feat_gender"].factorize()[0]
df  ['feat_manufacturer part number_cat'] = df["feat_manufacturer part number"].factorize()[0]
df  ['feat_material_cat'] = df["feat_material"].factorize()[0]

df  ['feat_sport_cat'] = df["feat_sport"].factorize()[0]
df  ['feat_style_cat'] = df["feat_style"].factorize()[0]

for key in keys:
  df[get_name_feat(key) + '_cat'] = df[get_name_feat(key)].factorize()[0]


In [82]:
df [ df.brand == df.feat_brand] [[ 'brand', 'feat_brand']].head()

,brand,feat_brand
2163,adidas,adidas
2417,adidas,adidas
3021,adidas,adidas
3054,adidas,adidas
3629,totes,totes


In [0]:
run_model(['brand_cat'])

In [83]:

df [ df.brand != df.feat_brand] [[ 'brand', 'feat_brand']].head()

,brand,feat_brand
0,Josmo,josmo
1,Josmo,josmo
2,SERVUS BY HONEYWELL,servus by honeywell
3,SERVUS BY HONEYWELL,servus by honeywell
4,SERVUS BY HONEYWELL,servus by honeywell


In [84]:
df['brand'] = df['brand'].map(lambda x: str(x).lower() )
df[ df.brand == df.feat_brand ].shape

(8846, 532)

In [0]:
feats = ['']

In [87]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat', 'feat_gender_cat', 'feat_manufacturer part number_cat', 'feat_material_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.16761631247025, 4.291874176588084)

In [88]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2608 ± 0.0112,brand_cat
0.1088 ± 0.0101,feat_material_cat
0.0415 ± 0.0039,feat_gender_cat
0.0219 ± 0.0013,feat_brand_cat
0.0055 ± 0.0002,feat_color_cat
0.0009 ± 0.0002,feat_manufacturer part number_cat


In [89]:
feats = ['brand_cat','feat_brand_cat','feat_color_cat', 'feat_gender_cat', 'feat_material_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.1220579961946, 4.247859392536408)

In [90]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2615 ± 0.0098,brand_cat
0.1025 ± 0.0116,feat_material_cat
0.0427 ± 0.0018,feat_gender_cat
0.0211 ± 0.0007,feat_brand_cat
0.0047 ± 0.0010,feat_color_cat


In [91]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.118891849994405, 4.170712624414773)

In [92]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2684 ± 0.0085,brand_cat
0.1096 ± 0.0132,feat_material_cat
0.0423 ± 0.0054,feat_gender_cat
0.0214 ± 0.0008,feat_brand_cat


In [93]:
df['brand'].value_counts(normalize=True)


nike                        0.097210
puma                        0.033315
ralph lauren                0.028775
vans                        0.021116
new balance                 0.020295
                              ...   
kirkland signature          0.000055
akadema                     0.000055
the felt store              0.000055
slipperooz by deer stags    0.000055
ganeshahandicraft           0.000055
Name: brand, Length: 1732, dtype: float64

In [96]:
df [ df['brand'] == 'nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [98]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_sport_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.13569216184724, 4.200972207191132)

In [99]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.09155058769064, 4.293801819292261)

In [100]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats)

Weight,Feature
0.2624 ± 0.0097,brand_cat
0.1070 ± 0.0091,feat_material_cat
0.0461 ± 0.0024,feat_gender_cat
0.0226 ± 0.0011,feat_brand_cat
0.0074 ± 0.0008,feat_style_cat


In [102]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.07893790688071, 4.313309830387125)

In [103]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)

perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats) 

Weight,Feature
0.2592 ± 0.0110,brand_cat
0.1055 ± 0.0082,feat_material_cat
0.0454 ± 0.0044,feat_gender_cat
0.0227 ± 0.0014,feat_brand_cat
0.0074 ± 0.0015,feat_style_cat
0.0003 ± 0.0000,feat_sport_cat


In [104]:
df [ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'style': 'basketball shoes', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'style': 'indoor soccer / casual', 'country/region of manufacture': 'china', 'condition': 'new with box'},
       {'sport': 'soccer', 'condition': 'new with box', 'type': 'cleats'},
       {'style': 'basketball shoes', 'condition': 'new with box', 'shade': 'red'},
       {'style': 'basketball shoes', 'pattern': 'solid', 'country/region of manufacture': 'china', 'condition': 'new with box'}],
      dtype=object)

In [105]:
df [ df['brand'] == 'nike'].features_parsed.sample(5).values

array([{'material': 'leather', 'gender': 'men', 'size': '12', 'color': 'white', 'model': '378336 120', 'manufacturer part number': '378336 120', 'brand': 'nike', 'age group': 'adult'},
       {'season': 'all-season', 'material': 'synthetic', 'gender': 'men', 'shoe size': '11', 'size': '11', 'color': 'black', 'model': '599525 001', 'manufacturer part number': '599525 001', 'brand': 'nike', 'age group': 'adult'},
       {'gender': 'men', 'brand': 'nike', 'manufacturer_part_number': '844626-001', 'fabric content': '100% fabric', 'color': 'black/black-white-anthracite'},
       {},
       {'sport': 'football', 'condition': 'new with box', 'type': 'cleats'}],
      dtype=object)

In [106]:
df['feat_age group'].value_counts()


adult               4563
men                  350
child                 77
men's                 33
unisex                 6
toddler                4
infant                 4
mens                   4
boys'                  3
youth                  2
women ,�� unisex       2
women                  2
men||women             2
adult ,�� teen         1
12 up                  1
Name: feat_age group, dtype: int64

In [110]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
#feats += feats_cat 
#feats = list(set(feats))

model= RandomForestRegressor(max_depth=5, n_estimators=100)
run_model(feats, model)

(-57.10868286663631, 4.3348102300962985)

In [0]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
#feats += feats_cat 
#feats = list(set(feats))

model= RandomForestRegressor(max_depth=5, n_estimators=100)
result = run_model(feats, model)

In [114]:
df['weight'].unique()

array([nan, '3.0 lbs', '9 g', '1.45 lbs', '0.45 lbs', '1.0 lbs',
       '0.23 lbs', '5.0 lbs', '5.5 lbs', '7.45 lbs', '4.0 lbs',
       '2.7969 lbs', '3.9 lbs', '4.6 pounds', '2.1 lbs', '1.1057 lbs',
       '15.0 lbs', '2.4 ounces', '454 g', '0.105 lbs', '9.1 ounces',
       '4.8 lbs', '6.1 lbs', '6.5 lbs', '1.1041 lbs', '1.3 Kg', '91 g',
       '20.0 lbs', '6.0 lbs', '386 g', '0.81 lbs', '4.5 lbs',
       '0.5 ounces', '2.0 lbs', '3.13 lbs', '5.9 lbs', '6.15 lbs',
       '1 pounds', '1.95 lbs', '2.15 lbs', '2 pounds', '2.1 pounds',
       '14 Kg', '0.4788 lbs', '10.0 lbs', '0.38 lbs', '2.5 lbs',
       '68.912 lbs', '45 g', '13.09 lbs', '2.5 pounds', '0.21 lbs',
       '16.75 lbs', '6.3 lbs', '272 g', '1.8 Kg', '2.8 pounds', '0.1 lbs',
       '5.05 lbs', '0.28 lbs', '76.08 lbs', '0.15 lbs', '200 g',
       '7.8 pounds', '399 g', '4.95 lbs', '64.144 lbs', '24 pounds',
       '73.696 lbs', '1.6 lbs', '6.6 ounces', '5 g', '1.2 Kg', '862 g',
       '3.05 lb', '8.6 ounces', '3.6 lbs', '71.

In [117]:
df['features'].unique()

array(['[{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["M"]},{"key":"Shoe Category","value":["Men\'s Shoes"]},{"key":"Color","value":["Multicolor"]},{"key":"Manufacturer Part Number","value":["8190-W-NAVY-7.5"]},{"key":"Brand","value":["Josmo"]}]',
       '[{"key":"Gender","value":["Men"]},{"key":"Color","value":["Black"]},{"key":"Shipping Weight (in pounds)","value":["0.45"]},{"key":"Condition","value":["New"]},{"key":"Brand","value":["SERVUS BY HONEYWELL"]},{"key":"manufacturer_part_number","value":["ZSR101BLMLG"]}]',
       '[{"key":"Heel Height","value":["Low (3/4 in. to 1 1/2 in.)"]},{"key":"Material","value":["Synthetic"]},{"key":"Gender","value":["Men"]},{"key":"Shoe Size","value":["10"]},{"key":"Fabric Material","value":["Synthetic"]},{"key":"Color","value":["Gray"]},{"key":"Casual & Dress Shoe Style","value":["Loafers & Slip Ons"]},{"key":"Manufacturer Part Number","value":["GLM02-077"]},{"key":"Brand","value":["NATIVE"]},{"key":"Age Group","value":["Adult"]},{"

In [118]:
feats_cat = [x for x in df.columns if '_cat' in x]
feats_cat

['brand_cat',
 'feat_catalog',
 'feat_brand_cat',
 'feat_color_cat',
 'feat_gender_cat',
 'feat_manufacturer part number_cat',
 'feat_material_cat',
 'feat_sport_cat',
 'feat_style_cat',
 'feat_combined shipping_cat',
 'feat_hairstyle_cat',
 'feat_series name_cat',
 'feat_uv protection_cat',
 'feat_lens material_cat',
 'feat_diamond color_cat',
 'feat_wheel type_cat',
 'feat_uv rating_cat',
 'feat_auto_cat',
 'feat_reinforced pockets_cat',
 'feat_country/region of manufacture_cat',
 'feat_band length_cat',
 'feat_character_cat',
 'feat_waterproof_cat',
 'feat_materials_cat',
 'feat_walmart no._cat',
 'feat_age_cat',
 'feat_mechanic_cat',
 'feat_product dimensions_cat',
 'feat_colour code_cat',
 'feat_product #_cat',
 'feat_product name_cat',
 'feat_assembled product weight_cat',
 'feat_height_cat',
 'feat_case finish_cat',
 'feat_frame depth_cat',
 'feat_frame material:_cat',
 'feat_removable hood_cat',
 'feat_cm_cat',
 'feat_nike id_cat',
 'feat_clothing product type_cat',
 'feat_prim

In [0]:
feats = ['brand_cat','feat_brand_cat', 'feat_gender_cat', 'feat_material_cat', 'feat_style_cat', 'feat_sport_cat']
feats += ['feat_combined shipping_cat',
 'feat_hairstyle_cat',
 'feat_series name_cat',
 'feat_uv protection_cat',
 'feat_lens material_cat',
 'feat_diamond color_cat',
 'feat_wheel type_cat',
 'feat_uv rating_cat',
 'feat_auto_cat',
 'feat_reinforced pockets_cat',
 'feat_country/region of manufacture_cat',
 'feat_band length_cat',
 'feat_character_cat',
 'feat_waterproof_cat',
 'feat_materials_cat',
 'feat_walmart no._cat',
 'feat_age_cat',
 'feat_mechanic_cat',
 'feat_product dimensions_cat',
 'feat_colour code_cat',
 'feat_product #_cat',
 'feat_product name_cat',
 'feat_assembled product weight_cat',
 'feat_height_cat',
 'feat_case finish_cat',
 'feat_frame depth_cat',
 'feat_frame material:_cat',
 'feat_removable hood_cat',
 'feat_cm_cat',
 'feat_nike id_cat',
 'feat_clothing product type_cat',
 'feat_primary color_cat',
 'feat_is polarized_cat',
 'feat_date first available_cat',
 'feat_date first available at amazon.co.uk_cat',
 'feat_boxed-product dimensions_cat',
 'feat_labor_cat',
 'feat_date_cat',
 'feat_sub type_cat',
 'feat_name_cat',
 'feat_water-resistant_cat',
 'feat_bezel_cat',
 'feat_polarised_cat',
 'feat_frame material_cat',
 'feat_licensed_cat',
 'feat_bracelet style_cat',
 'feat_leather grade_cat',
 'feat_hooded_cat',
 'feat_place of origin_cat',
 'feat_pattern_cat',
 'feat_number of wheels_cat',
 'feat_sock style_cat',
 'feat_lens type_cat',
 'feat_golf shoe type_cat',
 'feat_lined_cat',
 'feat_kids backpacks_cat',
 'feat_case diameter_cat',
 'feat_military style_cat',
 'feat_color family_cat',
 'feat_leg opening:_cat',
 'feat_watch shape_cat',
 'feat_resizable_cat',
 'feat_sub style_cat',
 'feat_video game platform_cat',
 'feat_protection_cat',
 'feat_sku number_cat',
 'feat_is energy star-certified_cat',
 'feat_comfort technology_cat',
 'feat_msrp_cat',
 'feat_elastic banded waist_cat',
 'feat_lens socket width_cat',
 'feat_audience_cat',
 'feat_age segment_cat',
 'feat_box_cat',
 'feat_stone type_cat',
 'feat_number of pieces_cat',
 'feat_eye size:_cat',
 'feat_band manufacturer_cat',
 'feat_fit_cat',
 'feat_number of items_cat',
 'feat_clasp type_cat',
 'feat_charge time (minutes)_cat',
 'feat_item color_cat',
 'feat_band color_cat',
 'feat_is water-resistant_cat',
 'feat_sleeve style_cat',
 'feat_date first available at amazon.ca_cat',
 'feat_wheeled_cat',
 'feat_материал_cat',
 'feat_general warranty_cat',
 'feat_airport friendly_cat',
 'feat_made in_cat',
 'feat_instrument_cat',
 'feat_recommended use_cat',
 'feat_product type_cat',
 'feat_shoe closure_cat',
 'feat_chest_cat',
 'feat_fabric care instructions_cat',
 'feat_country/ organization_cat',
 'feat_szie_cat',
 'feat_sizearm_cat',
 'feat_machine washable_cat',
 'feat_color/finish family_cat',
 'feat_crown_cat',
 'feat_safety features_cat',
 'feat_face shape_cat',
 'feat_leg_cat',
 'feat_power type_cat',
 'feat_manufacturer sku_cat',
 'feat_inv_cat',
 'feat_front style_cat',
 'feat_batteries included?_cat',
 'feat_conflict_cat',
 'feat_seller inventory number_cat',
 'feat_used with equipment type_cat',
 'feat_is wheeled_cat',
 'feat_disclosures_cat',
 'feat_era_cat',
 'feat_issued/ not-issued_cat',
 'feat_multi pack indicator_cat',
 'feat_item character_cat',
 'feat_shirt size_cat',
 'feat_length_cat',
 'feat_item spec attr - style (ebay)_cat',
 'feat_diameter_cat',
 'feat_watch power source_cat',
 'feat_hat style_cat',
 'feat_light weight_cat',
 'feat_use_cat',
 'feat_looks best on these face shapes:_cat',
 'feat_motion control_cat',
 'feat_mpn#_cat',
 'feat_case tone_cat',
 'feat_weather resistant_cat',
 'feat_colour_cat',
 'feat_high visibility (ansi compliant)_cat',
 'feat_applicable_cat',
 'feat_country////region of manufacture_cat',
 'feat_flame resistant_cat',
 'feat_auto shut-off_cat',
 'feat_style number_cat',
 'feat_is orthopedic_cat',
 'feat_case type:_cat',
 'feat_alarm_cat',
 'feat_suitable for_cat',
 'feat_item width_cat',
 'feat_product id_cat',
 'feat_expandable_cat',
 'feat_heat zones_cat',
 'feat_fine or fashion_cat',
 'feat_movement_cat',
 'feat_contains cubic zirconia_cat',
 'feat_style code_cat',
 'feat_number of heat settings_cat',
 'feat_sku_cat',
 'feat_removable liner_cat',
 'feat_compatible devices_cat',
 'feat_sports team_cat',
 'feat_shoe category_cat',
 'feat_assembled product dimensions (l x w x h)_cat',
 'feat_foot arch_cat',
 'feat_global composite sports type_cat',
 'feat_arm_cat',
 'feat_isbn_cat',
 'feat_chain length_cat',
 'feat_theme_cat',
 'feat_band width_cat',
 'feat_fits most screen size_cat',
 'feat_100% adidas authentic_cat',
 'feat_compass_cat',
 'feat_stylenumber_cat',
 'feat_euro size_cat',
 'feat_number of pockets_cat',
 'feat_attachment_cat',
 'feat_age group_cat',
 'feat_heel height_cat',
 'feat_display_cat',
 'feat_characteristics_cat',
 'feat_recommended location_cat',
 'feat_lining_cat',
 'feat_frame type_cat',
 'feat_hood_cat',
 'feat_fabric material_cat',
 'feat_age range_cat',
 'feat_item weight_cat',
 'feat_full product manual_cat',
 'feat_to fit_cat',
 'feat_frame color_cat',
 'feat_country//region of manufacture_cat',
 'feat_country of origin assembly:_cat',
 'feat_athlete_cat',
 'feat_safety apparel product type_cat',
 'feat_garment care_cat',
 'feat_vehicle type_cat',
 'feat_lining material_cat',
 'feat_has expiration_cat',
 'feat_purpose_cat',
 'feat_fit to_cat',
 'feat_occasion_cat',
 'feat_mirrored_cat',
 'feat_material composition_cat',
 'feat_material type_cat',
 'feat_outer material_cat',
 'feat_band material_cat',
 'feat_origin of components_cat',
 'feat_lens color family_cat',
 'feat_battery voltage_cat',
 'feat_lens tint_cat',
 'feat_fuel type_cat',
 'feat_nike product #_cat',
 'feat_chain/necklace length (in.)_cat',
 'feat_watch band material_cat',
 'feat_black_cat',
 'feat_quantity in set_cat',
 'feat_lens_cat',
 'feat_finish_cat',
 'feat_fabric_cat',
 'feat_type of sole_cat',
 'feat_navy blue_cat',
 'feat_spikes type_cat',
 'feat_hands_cat',
 'feat_resolution megapixels_cat',
 'feat_adidas_cat',
 'feat_water resistant_cat',
 'feat_hardsided or softsided_cat',
 'feat_shoe height_cat',
 'feat_frame shape_cat',
 'feat_manufacturer number_cat',
 'feat_upper material_cat',
 'feat_impact resistant_cat',
 'feat_closure_cat',
 'feat_product in inches (l x w x h)_cat',
 'feat_carats_cat',
 'feat_item package quantity_cat',
 'feat_metal color_cat',
 'feat_waist_cat',
 'feat_location - city/state_cat',
 'feat_is dark sky-compliant_cat',
 'feat_digital camera_cat',
 'feat_metal type_cat',
 'feat_chest size_cat',
 'feat_frame style_cat',
 'feat_light transmission (vlt)_cat',
 'feat_ean_cat',
 'feat_sole_cat',
 'feat_age gender group_cat',
 'feat_rise_cat',
 'feat_contains batteries_cat',
 'feat_diamond clarity_cat',
 'feat_includes_cat',
 'feat_designed for_cat',
 'feat_part number_cat',
 'feat_target audience_cat',
 'feat_gold karat_cat',
 'feat_safety feature_cat',
 'feat_autographed_cat',
 'feat_mpn_cat',
 'feat_designer_cat',
 'feat_hammer loop_cat',
 'feat_sock size_cat',
 'feat_year made_cat',
 'feat_insulation_cat',
 'feat_metal stamp_cat',
 'feat_sports league_cat',
 'feat_amazonbestsellersrank_cat',
 'feat_lens technology_cat',
 'feat_stability_cat',
 'feat_country of origin - assembly_cat',
 'feat_nickel free_cat',
 'feat_size/dimensions_cat',
 'feat_necklace type_cat',
 'feat_ring style_cat',
 'feat_jewelry setting_cat',
 'feat_contained battery type_cat',
 'feat_display technology_cat',
 'feat_bridge size:_cat',
 'feat_thick, warm, cozy_cat',
 'feat_animal type_cat',
 'feat_sizebridge_cat',
 'feat_bed size_cat',
 'feat_inseam_cat',
 'feat_lens color_cat',
 'feat_fabrication_cat',
 'feat_gps_cat',
 'feat_has adaptive lenses_cat',
 'feat_feature_cat',
 'feat_fabric care_cat',
 'feat_package_cat',
 'feat_type_cat',
 'feat_shade_cat',
 'feat_band length (in.)_cat',
 'feat_fits models_cat',
 'feat_bridge/temple size:_cat',
 'feat_fastener type_cat',
 'feat_jacket length_cat',
 'feat_casual & dress shoe style_cat',
 'feat_dial markers_cat',
 'feat_chronograph_cat',
 'feat_primary shelf id_cat',
 'feat_transactionid_cat',
 'feat_battery type_cat',
 'feat_neckline_cat',
 'feat_resolution_cat',
 'feat_power reserve_cat',
 'feat_construction_cat',
 'feat_material detail-1_cat',
 'feat_closure style_cat',
 'feat_fit:_cat',
 'feat_count_cat',
 'feat_shipping to usa_cat',
 'feat_temple length_cat',
 'feat_manufacturer_cat',
 'feat_style name_cat',
 'feat_certifications and listings_cat',
 'feat_authenticity_cat',
 'feat_clothing type_cat',
 'feat_country of manufacture_cat',
 'feat_country of origin_cat',
 'feat_size_cat',
 'feat_retail price_cat',
 'feat_type 2_cat',
 'feat_protects against_cat',
 'feat_width_cat',
 'feat_shoe fastening type_cat',
 'feat_reinforced knee_cat',
 'feat_has mercury_cat',
 'feat_enhanced visibility (reflective)_cat',
 'feat_insulated_cat',
 'feat_accessory type_cat',
 'feat_item style_cat',
 'feat_leather :_cat',
 'feat_what includes_cat',
 'feat_eyewear frame style_cat',
 'feat_front_cat',
 'feat_catalog_cat',
 'feat_style #_cat',
 'feat_shipping weight (in pounds)_cat',
 'feat_subdial_cat',
 'feat_model no._cat',
 'feat_casing_cat',
 'feat_cleaning, care & maintenance_cat',
 'feat_batteries required?_cat',
 'feat_is portable_cat',
 'feat_crystal_cat',
 'feat_number of compartments_cat',
 'feat_fits brands_cat',
 'feat_color mapping_cat',
 'feat_lens material:_cat',
 'feat_age start_cat',
 'feat_fits model_cat',
 'feat_domestic shipping_cat',
 'feat_profession_cat',
 'feat_case back_cat',
 'feat_case material_cat',
 'feat_sku#_cat',
 'feat_dimensions_cat',
 'feat_item type_cat',
 'feat_is waterproof_cat',
 'feat_number of batteries included_cat',
 'feat_heart rate monitor_cat',
 'feat_watch case shape_cat',
 'feat_eye protection type_cat',
 'feat_international shipping_cat',
 'feat_variant group id_cat',
 'feat_toe type_cat',
 'feat_item weight unit of measure_cat',
 'feat_release_cat',
 'feat_lens width_cat',
 'feat_wind resistant_cat',
 'feat_is lined_cat',
 'feat_ul safety listing_cat',
 'feat_overall material_cat',
 'feat_tactical_cat',
 'feat_clothing size_cat',
 'feat_boot height_cat',
 'feat_has paper wood_cat',
 'feat_fits styles_cat',
 'feat_technician_cat',
 'feat_polarized lenses_cat',
 'feat_pant style_cat',
 'feat_shape_cat',
 'feat_very popular bag now a days_cat',
 'feat_country of origin - components_cat',
 'feat_watch style_cat',
 'feat_genre_cat',
 'feat_measurements:_cat',
 'feat_is weather-resistant_cat',
 'feat_terrain_cat',
 'feat_pronation_cat',
 'feat_special features_cat',
 'feat_fishing_cat',
 'feat_article_cat',
 'feat_productgroup_cat',
 'feat_age end_cat',
 'feat_dial material_cat',
 'feat_packageweight_cat',
 'feat_country of origin components:_cat',
 'feat_handbag style_cat',
 'feat_label_cat',
 'feat_main color_cat',
 'feat_fabric type_cat',
 'feat_atpv arc rating_cat',
 'feat_pocket_cat',
 'feat_gemstone type_cat',
 'feat_temple size:_cat',
 'feat_bag size range_cat',
 'feat_model_cat',
 'feat_wash_cat',
 'feat_polarized_cat',
 'feat_model number_cat',
 'feat_band type_cat',
 'feat_fits over eyeglasses_cat',
 'feat_antiscratch lens coating_cat',
 'feat_features_cat',
 'feat_vendor description_cat',
 'feat_ring size_cat',
 'feat_cushioning_cat',
 'feat_footwear type_cat',
 'feat_free item_cat',
 'feat_bridge width_cat',
 'feat_features & fastening_cat',
 'feat_assembled in country of origin_cat',
 'feat_fastener_cat',
 'feat_interior pockets_cat',
 'feat_design_cat',
 'feat_net weight_cat',
 'feat_best sellers rank_cat',
 'feat_picture_cat',
 'feat_location - country_cat',
 'feat_chain included_cat',
 'feat_adjustable_cat',
 'feat_manufacturer_part_number_cat',
 'feat_stock no._cat',
 'feat_shoe size_cat',
 'feat_international shipping?_cat',
 'feat_guaranteed authentic_cat',
 'feat_season_cat',
 'feat_ground_cat',
 'feat_weather-resistant_cat',
 'feat_polarized?_cat',
 'feat_capacity_cat',
 'feat_batteries required_cat',
 'feat_condition_cat',
 'feat_rx-able_cat',
 'feat_fabric content_cat',
 'feat_main colour_cat',
 'feat_clothing size type_cat',
 'feat_����������������_cat',
 'feat_shoe width_cat',
 'feat_charger included_cat',
 'feat_waist size_cat',
 'feat_year_cat',
 'feat_style:_cat',
 'feat_country of manufacturer_cat',
 'feat_case type_cat',
 'feat_pattern or design_cat',
 'feat_hunting_cat',
 'feat_shipping weight_cat',
 'feat_sleeve length_cat',
 'feat_smart watch_cat',
 'feat_parts_cat',
 'feat_is recyclable_cat',
 'feat_clothing category_cat',
 'feat_part type_cat',
 'feat_work shoes_cat',
 'feat_case thickness_cat',
 'feat_dial color_cat',
 'feat_weight_cat',
 'feat_bridge_cat'
]


model= RandomForestRegressor(max_depth=5, n_estimators=100)
resultt= run_model(feats, model)

In [124]:
X= df[ feats ].values
y= df['prices_amountmin'].values

m= RandomForestRegressor(max_depth=5, n_estimators=100)
m.fit(X,y)
print(resultt)
perm = PermutationImportance(m, random_state=1).fit(X,y);
eli5.show_weights(perm, feature_names=feats) 

(-57.69822742857021, 4.214527148262217)


Weight,Feature
0.2478 ± 0.0069,brand_cat
0.1087 ± 0.0127,feat_material_cat
0.0136 ± 0.0025,feat_brand_cat
0.0124 ± 0.0013,feat_weight_cat
0.0110 ± 0.0012,feat_fabric content_cat
0.0069 ± 0.0027,feat_adjustable_cat
0.0061 ± 0.0008,feat_shoe category_cat
0.0053 ± 0.0013,feat_resizable_cat
0.0045 ± 0.0008,feat_gender_cat
0.0043 ± 0.0004,feat_fabric material_cat
